In [1]:
import api.bybit.REST_Ticker as bybit
import api.gate_io.API_Gate_io as gate_io
import api.htx.API_HTX as htx
import api.kukoin.API_Kukoin as kukoin
import threading
from queue import Queue
import pandas as pd
import datetime


cur_date = datetime.date.today()
cur_time = datetime.datetime.now().time().strftime("%H:%M")

q1 = Queue()
# q2 = Queue()
q3 = Queue()
q4 = Queue()    

# init threads
t1 = threading.Thread(target=bybit.get_data_from_endpoint, args=[q1, cur_date, cur_time])
# t2 = threading.Thread(target=gate_io.get_data_from_endpoint, args=[q2, cur_date, cur_time])
t3 = threading.Thread(target=htx.get_data_from_endpoint, args=[q3, cur_date, cur_time])
t4 = threading.Thread(target=kukoin.get_data_from_endpoint, args=[q4, cur_date, cur_time])

# start threads
t1.start()
# t2.start()
t3.start()
t4.start()

# join threads to the main thread
t1.join()
# t2.join()
t3.join()
t4.join()

q1 = q1.get()
# q2 = q2.get()
q3 = q3.get()
q4 = q4.get()

df1 = pd.DataFrame(q1)
df3 = pd.DataFrame(q3)
df4 = pd.DataFrame(q4)

frames = [df1, df3, df4]
df = pd.concat(frames)

In [40]:
df_r = df.assign(COUNT=df.groupby(by=['SYMBOL', 'CASH'])['PRICE'].transform('count'))
df_r = df_r[df_r.COUNT > 1]
df_r = df_r.assign(MIN=df_r.groupby(by=['SYMBOL', 'CASH'])['PRICE'].transform('min'))
df_r = df_r.assign(MAX=df_r.groupby(by=['SYMBOL', 'CASH'])['PRICE'].transform('max'))

In [41]:
df_r['DIFFER'] = df_r.apply(lambda row: row.MAX - row.MIN, axis = 1)
df_r = df_r.sort_values(by=['DIFFER', 'PRICE'], ignore_index=True, ascending=False)

In [42]:
df_r = df_r.iloc[:12]

In [43]:
df_r = df_r[['DATA', 'TIME', 'MARKET', 'SYMBOL', 'CASH', 'PRICE']]

In [44]:
df_r

,DATA,TIME,MARKET,SYMBOL,CASH,PRICE
0,2023-12-08,16:31,htx,BTC,BRL,239543.950000
1,2023-12-08,16:31,kukoin,BTC,BRL,216065.500000
2,2023-12-08,16:31,htx,BTC,GBP,36866.460000
3,2023-12-08,16:31,kukoin,BTC,GBP,34202.520000
4,2023-12-08,16:31,bybit,WBTC,USDT,43667.700000
5,2023-12-08,16:31,kukoin,WBTC,USDT,43310.700000
6,2023-12-08,16:31,kukoin,GRAIL,USDT,1567.020000
7,2023-12-08,16:31,htx,GRAIL,USDT,1462.410000
8,2023-12-08,16:31,kukoin,QUICK,USDT,58.790000
9,2023-12-08,16:31,htx,QUICK,USDT,0.052074
